In [18]:
!pip install pandas
import pandas as pd
import re

**Cargar los datos. Revisa que cada tipo de dato esté cargado correctamente. Si
es numérico, que sea de esa forma, si es fecha, que sea de tipo de fecha, etc.**

In [19]:
try:
  df_viajes = pd.read_csv('https://do-strapi.s3.us-east-1.amazonaws.com/viajes_dd8a0ac9e2.csv')
  print("Viajes ok")
except Exception as e:
  print(f"Error importando viajes: {e}")

try:
  df_mediciones = pd.read_csv('https://do-strapi.s3.us-east-1.amazonaws.com/mediciones_4be6910e87.csv')
  print("Mediciones ok")
except Exception as e:
  print(f"Error importando medisiones: {e}")

#asegurandose que sea fecha:
df_viajes['yyyy-mm-ddThh:mm:ss.sss'] = pd.to_datetime(df_viajes['yyyy-mm-ddThh:mm:ss.sss'])

Viajes ok
Mediciones ok


In [8]:
print(df_viajes.info())
print(df_mediciones.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        59 non-null     int64  
 1   Cruise                    59 non-null     object 
 2   Station                   59 non-null     float64
 3   Type                      59 non-null     object 
 4   yyyy-mm-ddThh:mm:ss.sss   59 non-null     object 
 5   Longitude [degrees_east]  59 non-null     float64
 6   Latitude [degrees_north]  59 non-null     float64
 7   Bot. Depth [m]            59 non-null     float64
 8   Ship                      59 non-null     object 
dtypes: float64(4), int64(1), object(4)
memory usage: 4.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100400 entries, 0 to 100399
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id         100400 non-null  i

In [5]:
print(df_viajes.head())
print(df_mediciones.head())

   id        Cruise  Station Type yyyy-mm-ddThh:mm:ss.sss  \
0  18  Crucero 2016     37.0    C     2016-10-25 12:10:00   
1  14  Crucero 2016     26.0    C     2016-10-23 16:13:00   
2  15  Crucero 2016     27.0    C     2016-10-23 21:17:00   
3  16  Crucero 2016     28.0    C     2016-10-23 22:59:00   
4  17  Crucero 2016     29.0    C     2016-10-24 00:46:00   

   Longitude [degrees_east]  Latitude [degrees_north]  Bot. Depth [m] Ship  
0                  73.05760                  46.17430           270.0   A1  
1                  72.01045                  46.23823           260.0   A1  
2                  72.18855                  46.27144           222.0   A1  
3                  72.34764                  46.28825           228.0   A1  
4                  72.51546                  46.26607           212.0   A1  
   id  Depth [m]                                    variable  valor
0  18      0.497  Fluorescence, WET Labs ECO-AFL/FL [mg/m^3]    0.0
1  18      1.010  Fluorescence, WET

**Unir ambos archivos usando la columna “id”.**

In [15]:

df_unido = pd.merge(df_viajes, df_mediciones, on='id', how='outer')
print(df_unido.head())
print(df_unido.info())

   id        Cruise  Station Type yyyy-mm-ddThh:mm:ss.sss  \
0   0  Crucero 2016      1.0    C     2016-10-19 22:15:00   
1   0  Crucero 2016      1.0    C     2016-10-19 22:15:00   
2   0  Crucero 2016      1.0    C     2016-10-19 22:15:00   
3   0  Crucero 2016      1.0    C     2016-10-19 22:15:00   
4   0  Crucero 2016      1.0    C     2016-10-19 22:15:00   

   Longitude [degrees_east]  Latitude [degrees_north]  Bot. Depth [m] Ship  \
0                  73.48573                  46.22015           485.0   A1   
1                  73.48573                  46.22015           485.0   A1   
2                  73.48573                  46.22015           485.0   A1   
3                  73.48573                  46.22015           485.0   A1   
4                  73.48573                  46.22015           485.0   A1   

   Depth [m]                                    variable  valor  
0      0.557  Fluorescence, WET Labs ECO-AFL/FL [mg/m^3]    0.0  
1      1.462  Fluorescence, WET 

**Estandariza las fechas. Recuerda que para una misma estación se asume que el
tiempo no varía. Como consecuencia, se requiere mantener la primera fecha
observada para cada combinación de estación y crucero.**

In [16]:

df_unido['Fecha_Estandarizada'] = df_unido.groupby(['Cruise', 'Station'])['yyyy-mm-ddThh:mm:ss.sss'].transform('min')
df_unido.drop(columns=['yyyy-mm-ddThh:mm:ss.sss'], inplace=True)
df_unido.rename(columns={'Fecha_Estandarizada': 'yyyy-mm-ddThh:mm:ss.sss'}, inplace=True)


print(df_unido.head())
print(df_unido.info())

   id        Cruise  Station Type  Longitude [degrees_east]  \
0   0  Crucero 2016      1.0    C                  73.48573   
1   0  Crucero 2016      1.0    C                  73.48573   
2   0  Crucero 2016      1.0    C                  73.48573   
3   0  Crucero 2016      1.0    C                  73.48573   
4   0  Crucero 2016      1.0    C                  73.48573   

   Latitude [degrees_north]  Bot. Depth [m] Ship  Depth [m]  \
0                  46.22015           485.0   A1      0.557   
1                  46.22015           485.0   A1      1.462   
2                  46.22015           485.0   A1      2.007   
3                  46.22015           485.0   A1      2.523   
4                  46.22015           485.0   A1      3.026   

                                     variable  valor yyyy-mm-ddThh:mm:ss.sss  
0  Fluorescence, WET Labs ECO-AFL/FL [mg/m^3]    0.0     2016-10-19 22:15:00  
1  Fluorescence, WET Labs ECO-AFL/FL [mg/m^3]    0.0     2016-10-19 22:15:00  
2  Fl

**Estandariza los nombres de las variables (nombre de las columnas)**

In [27]:
def limpiar_columnas(col_name):
    col_name = col_name.lower()
    col_name = re.sub(r'[^a-z0-9_-]', '_', col_name)
    return col_name

df_unido.columns = [limpiar_columnas(col) for col in df_unido.columns]

print(df_unido.head())
print(df_unido.info())

   id        cruise  station type  longitude__degrees_east_  \
0   0  Crucero 2016      1.0    C                  73.48573   
1   0  Crucero 2016      1.0    C                  73.48573   
2   0  Crucero 2016      1.0    C                  73.48573   
3   0  Crucero 2016      1.0    C                  73.48573   
4   0  Crucero 2016      1.0    C                  73.48573   

   latitude__degrees_north_  bot__depth__m_ ship  depth__m_  \
0                  46.22015           485.0   A1      0.557   
1                  46.22015           485.0   A1      1.462   
2                  46.22015           485.0   A1      2.007   
3                  46.22015           485.0   A1      2.523   
4                  46.22015           485.0   A1      3.026   

                               variable  valor yyyy-mm-ddthh_mm_ss_sss  
0  Fluorescence WET Labs ECO-AFLFL mgm3    0.0     2016-10-19 22:15:00  
1  Fluorescence WET Labs ECO-AFLFL mgm3    0.0     2016-10-19 22:15:00  
2  Fluorescence WET Lab

**Aplique la misma estandarización del punto 3 para los valores de la columna
“variable”**

In [28]:

crucero_mas_datos = df_unido.groupby('cruise')['id'].count().idxmax()
print(f"El crucero con más datos es: {crucero_mas_datos}")


El crucero con más datos es: Crucero 2017


In [29]:

df_unido['hora'] = df_unido['yyyy-mm-ddthh_mm_ss_sss'].dt.hour

cuenta_hora = df_unido['hora'].value_counts()
hora_con_mas = cuenta_hora.idxmax()

print(f"La hora del día con más muestreos considerando los 3 años es: {hora_con_mas}")


La hora del día con más muestreos considerando los 3 años es: 20
